# Modélisation 

In [170]:
# Functions imports

import tqdm as notebook_tqdm

import numpy as np
import importlib
import functions
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

from skopt import BayesSearchCV
from skopt.space import Real, Integer

import xgboost as xgb

importlib.reload(functions)
from functions import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [171]:
x = pd.read_csv("../data/processed/x_processed.csv")  # 10605 rows × 10 columns
x_challenge = pd.read_csv("../data/processed/x_challenge_processed.csv")  # 4942 rows × 10 columns
x_challenge_unprocessed = pd.read_csv("../data/raw/X_test_GgyECq8.csv")
y = pd.read_csv("../data/raw/y_train_jJtXgMX.csv")  # 10605 rows × 2 columns

In [172]:
y.head(5)

,DELIVERY_START,spot_id_delta
0,2022-01-01 02:00:00+01:00,-36.874770
1,2022-01-01 03:00:00+01:00,-12.643588
2,2022-01-01 04:00:00+01:00,-1.950193
3,2022-01-01 05:00:00+01:00,1.938272
4,2022-01-01 06:00:00+01:00,0.199907


Préparation des données 

In [173]:
x_train, x_test, y_train, y_test = train_test_split(x.drop(columns=['DELIVERY_START']), y, test_size=.2)

# y_bin = pd.DataFrame({
#     'spot_id_delta' : (y['spot_id_delta'] > 0).astype(int)
# })

# y_abs = pd.DataFrame({
#     'spot_id_delta' : abs(y['spot_id_delta'])
# })

# y_bin.head(10) # Quand c'est négatif c'est 0 et quand c'est positif c'est 1 (Signe à prévoir en classfication)
# y_abs.head(10) # La valeur absolue à prévoir en régression.

In [174]:
# Normalisation (inutile pour les arbres de décisions)

# scaler = StandardScaler()
# x = scaler.fit_transform(x)

# x = pd.DataFrame(x) # scaler renvoie des numpy.darray

In [175]:
forest = RandomForestRegressor(
    n_estimators=200,
    # min_samples_split=2, 
    # min_samples_leaf=1,
    # max_features='sqrt',
    # max_depth=40,
    # bootstrap=False,
    # random_state=42
)

boost = xgb.XGBRegressor(
    # subsample=0.9,
    # objective='reg:squarederror',
    # n_estimators=300,
    # min_child_weight=3,
    # max_depth=7,
    # learning_rate=0.1,
    # reg_lambda=2,
    # gamma=0.1,
    # colsample_bytree=0.7,
    # reg_alpha=0.1
)



# forest_bin = RandomForestClassifier( # La RSCV n'a pas amélioré les scores, peut-être la refaire
#     n_estimators=200,
#     min_samples_split=10, 
#     min_samples_leaf=4,
#     max_features='log2',
#     max_depth=50,
#     bootstrap=False,
#     random_state=42
# )

Cross validation pour le modèle de regression

In [176]:
# cross_validation(forest, x_train, y_train['spot_id_delta'], 5, 'neg_mean_squared_error')

In [177]:
cross_validation(boost, x_train, y_train['spot_id_delta'], 5, 'neg_mean_squared_error')

##### CROSS-VALIDATION SCORES ##### 

 neg_mean_squared_error  scores from cross-validation  [ -589.4831233   -501.11242257  -437.66255701  -430.88141186
 -3234.34439367] 
 neg_mean_squared_error  mean from cross-validation  -1038.6967816806005 
 Standard deviation  1099.3034115362123 

 ###################################


Cross validation pour le modèle de classification

In [178]:
# cross_validation(forest_bin, x, y_bin['spot_id_delta'], 5, 'accuracy')

In [179]:
# En résumé, les résultats ne sont pas mauvais. j'ai juste 9 en std donc je vais faire la RSCV pour améliorer cela.

# Optimisation

In [180]:
# # J'optimise avec une RSCV

# # Paramètres pour la RandomizedSearchCV

# param_dist_forest = {
#     'n_estimators': [int(x) for x in np.linspace(start=100, stop=1200, num=12)],
#     'max_features': [None, 'sqrt', 'log2'],
#     'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# forest_random_search = RandomizedSearchCV (
#     estimator=forest,
#     param_distributions=param_dist_forest,
#     n_iter=100,
#     cv=3,
#     verbose=2,
#     n_jobs=-1
# )

# forest_random_search.fit(x_train, y_train['spot_id_delta'])

# print(f"Meilleurs paramètres : {forest_random_search.best_params_}")

# # Meilleurs paramètres pour le regressor : {'n_estimators': 400, 'min_samples_split': 2,
# #  'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}

In [181]:
# # Je fais une GridSearchCV pour le XGBoost. C'est exploratoire, je ne vois pas pourquoi ce serait mieux que la RSCV.

# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [3, 5, 7],
#     'subsample': [0.7, 0.8, 0.9],
#     'colsample_bytree': [0.7, 0.8, 0.9],
#     'gamma': [0, 0.1, 0.2],
#     'min_child_weight': [1, 3, 5],
#     'reg_alpha': [0, 0.1, 0.5],
#     'reg_lambda': [1, 1.5, 2]
# }

# grid_search = GridSearchCV(
#     estimator=boost, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1
# )

# grid_search.fit(x_train, y_train['spot_id_delta'])
# best_params = grid_search.best_params_
# print("Meilleurs paramètres pour la GridSearchCV: ", best_params)

In [182]:
# # Je fais une bayes search parce que c'est ce qui a fonctionné pour le CFM.

# param_grid = {
#     'n_estimators': Integer(100, 1000),
#     'max_depth': Integer(3, 10),
#     'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
#     'subsample': Real(0.6, 1.0),
#     'colsample_bytree': Real(0.6, 1.0),
#     'min_child_weight': Integer(1, 10),
#     'gamma': Real(0.0, 0.5),
#     'reg_alpha': Real(0.0, 1.0),
#     'reg_lambda': Real(0.0, 1.0)
# }

# # Configuration de la recherche bayésienne
# bayes_search = BayesSearchCV(
#     estimator=boost,
#     search_spaces=param_grid,
#     n_iter=50,
#     scoring='neg_mean_squared_error',
#     cv=3,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42
# )

# bayes_search.fit(x_train, y_train['spot_id_delta'])

# print("Meilleurs hyperparamètres :", bayes_search.best_params_)

Prédiction

In [183]:
forest.fit(x_train, y_train['spot_id_delta'])
# forest_bin.fit(x, y_bin['spot_id_delta'])

# metrics(forest, x_test, y_test, 'Métriques pour la première RandomForest', 'regressor')

RandomForestRegressor(n_estimators=200)

In [184]:
# Je fais une weighted accuracy pour pouvoir évaluer moi-même les performances du modèle 
# au lieu de devoir soumettre sur le site de l'ENS.

def weighted_accuracy(y_true, y_pred, weights):

    # Déterminer les prédictions correctes en termes de signe
    correct_predictions = np.sign(y_true) == np.sign(y_pred)
    
    # Calculer la weighted accuracy
    weighted_acc = np.sum(correct_predictions * weights) / np.sum(weights)
    return weighted_acc

In [185]:
weights = np.abs(y_test['spot_id_delta'])

y_pred = forest.predict(x_test)

# Évaluation
mse = mean_squared_error(y_test['spot_id_delta'], y_pred)
weighted_accc = weighted_accuracy(y_test['spot_id_delta'], y_pred, weights)

print(f"Mean squarred error : {mse:.2f}") # Les prédictions sont proches des valeurs réelles ou non
print(f"Weighted accuracy : {weighted_accc:.2f}") # Qualité à prédire les signes sur les valeurs importantes

# Vis à vis de cette sortie, forest est bien meilleure que boost

Mean squarred error : 647.73
Weighted accuracy : 0.91


In [186]:
y_challenge = pd.DataFrame({ # Faut prendre 'x_challenge_unprocessed' parce que sinon y'a le décalage horaire
    'DELIVERY_START' : x_challenge_unprocessed['DELIVERY_START']  
    })

y_challenge['spot_id_delta'] = forest.predict(x_challenge.drop(columns=['DELIVERY_START']))
# y_pred['spot_id_delta_bin'] = forest_bin.predict(x_challenge)


y_challenge.head(5)

,DELIVERY_START,spot_id_delta
0,2023-04-02 00:00:00+02:00,9.247896
1,2023-04-02 01:00:00+02:00,10.144571
2,2023-04-02 02:00:00+02:00,-1.008952
3,2023-04-02 03:00:00+02:00,0.013697
4,2023-04-02 04:00:00+02:00,-0.979499


In [187]:
# y_pred['spot_id_delta_bin'] = y_pred['spot_id_delta_bin'].apply(lambda x: 1 if x==1 else -1)

# y_pred['spot_id_delta'] = y_pred['spot_id_delta_abs'] * y_pred['spot_id_delta_bin']
# y_pred = y_pred.drop(columns=['spot_id_delta_abs', 'spot_id_delta_bin'])

# y_pred.head(5)

In [188]:
y_challenge.to_csv("../data/predicted/y_challenge_15.csv", index=False)

In [ ]:
# J'ai toujours le même problème sur ce challenge. J'ai beau entraîner des modèles de plus en plus complexes,
# les résultats ne sont pas plus concluants car la corrélation entre les données d'entraînement et les données 
# de soumission n'est pas la même. Toutes les relations apprises par mes algorithmes lors de l'entraînement, sont 
# en fait inutile pour la prédiction sur les données de soumission. 

# C'est un problème d'overfitting que je n'arrive pas à régler en pré-traitement ou en prédiction. 